In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from huggingface_hub import login, notebook_login
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
login(token=os.getenv("HUGGINGFACE_TOKEN"))

In [3]:
checkpoint = "GaaS-Team/DistilBERT-finetuned-GaaS"  # Example checkpoint for sentiment analysis
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

raw_text = [
    "10/10, my PC begged for mercy within 5 minutes. This game is the ultimate hardware stress test.",
    "Love the immersive experience of staring at a loading screen for 20 minutes just to get yeeted from the server. Devs, you’ve outdone yourselves.",
    "The bugs are so next-level, I’m pretty sure this game is secretly a sci-fi horror sim. Where’s my Oscar for surviving this mess?",
    "Graphics so retro, my eyes are still adjusting to the 90s CRT monitor vibes. Truly a masterpiece of aesthetic suffering.",
    "Didn’t realize ‘pay-to-win’ was a whole genre until I dropped $50 and still got owned. Take my money, I guess?",
    "The storyline is so gripping, I fell asleep trying to find it. 10/10 for accidental ASMR.",
    "Perfect game for anyone who loves slamming their keyboard in frustration. Next-level rage bait, devs. Respect.",
    "That 5-second tutorial was chef’s kiss. I’m still figuring out how to move, but who needs guidance, right?",
    "The lag is so consistent, it’s basically a core mechanic. I’m speedrunning disconnects now.",
    "I’d ask for a refund, but I’m too busy uninstalling this work of art. Devs, you’ve earned my eternal respect and my Alt+F4."
]

inputs = tokenizer(raw_text, padding=True, truncation=True, return_tensors="pt")

# Run inference
with torch.no_grad():
    logits = model(**inputs).logits

# Get predicted classes
predicted_class_ids = logits.argmax(dim=-1).tolist()
predicted_classes = [model.config.id2label[id] for id in predicted_class_ids]

# Get probabilities
probs = torch.nn.functional.softmax(logits, dim=-1).numpy()

# Print results for each review
for i, (text, pred_class, prob) in enumerate(zip(raw_text, predicted_classes, probs)):
    print(f"Review {i+1}: {text}")
    print(f"Predicted class: {pred_class}")
    print(f"Probabilities: {prob}")
    print()

Review 1: 10/10, my PC begged for mercy within 5 minutes. This game is the ultimate hardware stress test.
Predicted class: POSITIVE
Probabilities: [0.00929471 0.9907053 ]

Review 2: Love the immersive experience of staring at a loading screen for 20 minutes just to get yeeted from the server. Devs, you’ve outdone yourselves.
Predicted class: NEGATIVE
Probabilities: [0.76630455 0.23369542]

Review 3: The bugs are so next-level, I’m pretty sure this game is secretly a sci-fi horror sim. Where’s my Oscar for surviving this mess?
Predicted class: NEGATIVE
Probabilities: [0.99453074 0.00546924]

Review 4: Graphics so retro, my eyes are still adjusting to the 90s CRT monitor vibes. Truly a masterpiece of aesthetic suffering.
Predicted class: POSITIVE
Probabilities: [0.01362654 0.9863734 ]

Review 5: Didn’t realize ‘pay-to-win’ was a whole genre until I dropped $50 and still got owned. Take my money, I guess?
Predicted class: NEGATIVE
Probabilities: [0.990266   0.00973399]

Review 6: The stor